how this data contribution expands the OBIS holdings

- picking a species from SCCOOS, then pulling all the data for that species from OBIS
- plotting it all together with SCCOOS on a timeslider.

In [1]:
# get SCCOOS occurrence and measurement or fact data
library(robis)
library(leaflet)

sccoos_dataset_id = "c9aaa0e9-8f6c-4553-a014-a857baba0680"

species = "Lingulodinium polyedra"

sccoos_occur <- occurrence(
    datasetid=sccoos_dataset_id, mof=TRUE
)

Retrieved 5000 records of approximately 10483 (47%)
Retrieved 10000 records of approximately 10483 (95%)
Retrieved 10483 records of approximately 10483 (100%)


In [2]:
# make table bigger for review
options(repr.matrix.max.cols=100)

# collect column names and drop 'mof' from list:
cols <- colnames(sccoos_occur)
cols <- cols[cols!='mof']

# One occurrence observation without measurement or fact data:
head(sccoos_occur[cols],n = 1)

infraphylum,date_year,scientificNameID,scientificName,dropped,hab,aphiaID,decimalLatitude,infraphylumid,phylumid,familyid,occurrenceStatus,basisOfRecord,subkingdom,organismName,maximumDepthInMeters,id,order,organismQuantity,dataset_id,decimalLongitude,date_end,occurrenceID,infrakingdom,subkingdomid,date_start,organismQuantityType,genus,eventDate,eventID,infrakingdomid,coordinateUncertaintyInMeters,absence,eventRemarks,genusid,taxonID,originalScientificName,marine,minimumDepthInMeters,subphylumid,date_mid,class,orderid,geodeticDatum,kingdom,classid,phylum,locationID,subphylum,family,kingdomid,node_id,sss,depth,shoredistance,sst,bathymetry,subclassid,terrestrial,speciesid,superorder,superorderid,wrims,species,subclass,flags,brackish
<chr>,<int>,<chr>,<chr>,<lgl>,<lgl>,<int>,<dbl>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<lgl>,<chr>,<int>,<chr>,<chr>,<lgl>,<int>,<int>,<dbl>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<lgl>,<int>,<chr>,<int>,<lgl>,<chr>,<chr>,<chr>,<lgl>
Dinoflagellata,2018,(urn:lsid:marinespecies.org:taxname:109470),Alexandrium,FALSE,TRUE,109470,34.408,146203,450030,109431,present,HumanObservation,Harosa,Alexandrium_spp,1,00024365-b76d-4722-9530-538372d53272,Gonyaulacales,11184,c9aaa0e9-8f6c-4553-a014-a857baba0680,-119.685,1.526256e+12,HABs-StearnsWharf_2018-05-14T15:51:00Z_Alexandrium_spp,Alveolata,582419,1.526256e+12,cells/L,Alexandrium,2018-05-14T15:51:00Z,HABs-StearnsWharf_2018-05-14T15:51:00Z,536209,12,FALSE,20180514,109470,urn:lsid:algaebase.org:taxname:6378,Alexandrium,TRUE,1,562620,1.526256e+12,Dinophyceae,109391,EPSG:4326 WGS84,Chromista,19542,Myzozoa,HABs-StearnsWharf,Dinozoa,Ostreopsidaceae,7,b7c47783-a020-4173-b390-7b57c4fa1426,33.44,1,202,15.4,7,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [3]:
# additional observations (measurements or facts) connected to the occurrence above
sccoos_mof <- sccoos_occur$mof
head(sccoos_mof,n = 1)

,measurementDeterminedBy,measurementAccuracy,measurementValue,measurementRemarks,measurementValueID,level,occurrenceID,measurementUnit,measurementDeterminedDate,measurementType,measurementUnitID,measurementTypeID,measurementID,measurementMethod
,<lgl>,<lgl>,<chr>,<lgl>,<lgl>,<int>,<lgl>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<lgl>
1,NA,NA,6.728,NA,NA,1,NA,uM,NA,Ammonium,NA,http://vocab.nerc.ac.uk/collection/S27/current/CS026908/,HABs-StearnsWharf_2018-05-14T15:51:00Z_Ammonium,NA
2,NA,NA,15,NA,NA,1,NA,Celsius,NA,Temp,http://vocab.nerc.ac.uk/collection/P06/current/UPAA/,http://vocab.nerc.ac.uk/collection/P25/current/WTEMP/,HABs-StearnsWharf_2018-05-14T15:51:00Z_Temp,NA
3,NA,NA,7.393,NA,NA,1,NA,uM,NA,Silicate,NA,"http://vocab.nerc.ac.uk/collection/P25/current/SLCA/ , https://www.bco-dmo.org/parameter/1209",HABs-StearnsWharf_2018-05-14T15:51:00Z_Silicate,NA
4,NA,NA,0.764,NA,NA,1,NA,uM,NA,Phosphate,NA,http://vocab.nerc.ac.uk/collection/S27/current/CS026904/,HABs-StearnsWharf_2018-05-14T15:51:00Z_Phosphate,NA
5,NA,NA,0.903,NA,NA,1,NA,mg/m3,NA,Phaeo1,http://vocab.nerc.ac.uk/collection/P06/current/UMMC/,"http://vocab.nerc.ac.uk/collection/P25/current/PHAEO/ , https://www.bco-dmo.org/parameter/925 , http://vocab.nerc.ac.uk/collection/P01/current/PHAEFMP1/",HABs-StearnsWharf_2018-05-14T15:51:00Z_Phaeo1,NA
6,NA,NA,0,NA,NA,1,NA,ng/mL,NA,pDA,NA,https://pubchem.ncbi.nlm.nih.gov/compound/Domoic-acid,HABs-StearnsWharf_2018-05-14T15:51:00Z_pDA,NA
7,NA,NA,0.368,NA,NA,1,NA,uM,NA,Nitrite_Nitrate,NA,http://vocab.nerc.ac.uk/collection/S27/current/CS026905/,HABs-StearnsWharf_2018-05-14T15:51:00Z_Nitrite_Nitrate,NA
8,NA,NA,0,NA,NA,1,NA,uM,NA,Nitrite,NA,"http://vocab.nerc.ac.uk/collection/S27/current/CS002877/ , https://www.bco-dmo.org/parameter/1192 , http://vocab.nerc.ac.uk/collection/P02/current/NTRI/",HABs-StearnsWharf_2018-05-14T15:51:00Z_Nitrite,NA
9,NA,NA,0.368,NA,NA,1,NA,uM,NA,Nitrate,NA,"http://vocab.nerc.ac.uk/collection/S27/current/CS002879/ , https://www.bco-dmo.org/parameter/1194 ,",HABs-StearnsWharf_2018-05-14T15:51:00Z_Nitrate,NA


In [4]:
# create a map
library(htmlwidgets)
library(IRdisplay)

m = map_leaflet(sccoos_occur)

saveWidget(m, "sccoos_data.html", selfcontained = FALSE)
display_html("<iframe src=\"sccoos_data.html\"></iframe>")

In [5]:
# get all Lingulodinium polyedra occurrences from OBIS
lingpoly_occur <- occurrence(
    scientificname = species
)

# or by taxon id from AphiaID: urn:lsid:algaebase.org:taxname:233592
# could use rtaxize 'get_wormsid' here or obistools 'match_taxa' to search on taxonid from species name? 
# lingpoly_occur <- occurrence(
#   taxonid = '233592'
# )

Retrieved 5000 records of approximately 9199 (54%)
Retrieved 9199 records of approximately 9199 (100%)


In [6]:
# remove sccoos entries from full occurrence response
lingpoly_occur <- lingpoly_occur[!lingpoly_occur$dataset_id==sccoos_dataset_id,]

In [7]:
# create a map of all lingulodinium polyedra, excluding sccoos contributions
m = map_leaflet(lingpoly_occur)

saveWidget(m, "lingpoly_data.html", selfcontained = FALSE)
display_html("<iframe src=\"lingpoly_data.html\"></iframe>")

In [9]:
# subset sccoos observations for Lingulodinium and add those markers to global map
sccoos_lingpoly_occur <- sccoos_occur[ which(sccoos_occur$scientificName == species), ]

m = map_leaflet(lingpoly_occur) %>% 
  addTiles() %>% 
  addMarkers(
      sccoos_lingpoly_occur$decimalLongitude, sccoos_lingpoly_occur$decimalLatitude,
      clusterOptions = markerClusterOptions(),
    #radius = 3, weight = 1, opacity = 1, fillOpacity = 0.1, 
      popup = sccoos_lingpoly_occur$mof[[1]]
  )


saveWidget(m, "lingpoly_data_sub.html", selfcontained = FALSE)
display_html("<iframe src=\"lingpoly_data_sub.html\"></iframe>")

In [ ]:
# print occurrenceID's from mof observations for the first occurrence 
sccoos_lingpoly_occur$mof[[1]]$measurementValue

In [11]:
# TODO
# make the popup's on the map above have a table of the mof observations for each occurrence.
# Might need to have the popup with a time slider since the SCCOOS occurrences are at only 7 locations for 1007 occurrences.
m = map_leaflet(lingpoly_occur) %>% 
  addTiles() %>% 
  addMarkers(
    sccoos_lingpoly_occur$decimalLongitude, sccoos_lingpoly_occur$decimalLatitude,
    clusterOptions = markerClusterOptions(),
    #radius = 3, weight = 1, opacity = 1, fillOpacity = 0.1, 
    popup = ~paste("<table>
                        <thead><tr><td>measurementType</td><td>measurementValue</td></tr></thead>
                        <tbody><tr><td>", sccoos_lingpoly_occur$mof[[1]]$measurementType, "</td><td>", sccoos_lingpoly_occur$mof[[1]]$measurementValue, "</td></tr></tbody>
                    </table>")
  )

saveWidget(m, "lingpoly_data_sub.html", selfcontained = FALSE)
display_html("<iframe src=\"lingpoly_data_sub.html\"></iframe>")